In [17]:
###Rather than reinitializing maybe update keys
class keys:
    def __init__(self, event= None):
        self.last_pressed = ""
        self.string = "Pressed:"
        if event.key == pygame.K_UP:
            self.UP = True
            self.string += "UP"
        else:
            self.UP = False
        if event.key == pygame.K_DOWN:
            self.DOWN = True
            self.string += "DOWN"
        else:
            self.DOWN = False
        if event.key == pygame.K_RIGHT:
            self.RIGHT = True
            self.string += "RIGHT"
            self.last_pressed = "RIGHT"
        else:
            self.RIGHT = False    
        if event.key == pygame.K_LEFT:
            self.LEFT = True
            self.string += "LEFT"
            self.last_pressed = "LEFT"
        else:
            self.LEFT = False 
 
    def __str__(self):
        return self.string
        
        
def xy_position_to_pixels(Rectangle, window_size, virtual_window):
    """Takes:In-game coordinate rectangle
    Converts in game coordinates to pixels. Also, note that the in-game position refers to the LOWER RIGHT hand
    corner of the sprite. 
    Returns: rectangle in pixel coordinates"""
    x = Rectangle[0]
    y = Rectangle[1]
    width = Rectangle[2]
    height = Rectangle[3]
    pixel_x = int(float(x/virtual_window[0])*window_size[0])
    pixel_y = window_size[1] - int(float(y/virtual_window[1])*window_size[1])
    pixel_width = int(float(width/virtual_window[0])*window_size[0])
    pixel_height = int(float(height/virtual_window[1])*window_size[1])
    pixel_y -= pixel_height
    return (pixel_x, pixel_y, pixel_width, pixel_height)



class position:
    def __init__(self, current_x = 0, current_y = 0, current_xs = 0, current_ys = 0, floor = 0, button=None,
                character_width = 20, character_height = 30, wall_speed = .3):
        self.current_x = current_x
        self.current_y = current_y
        self.current_xs = current_xs
        self.current_ys = current_ys
        self.button = button
        self.floor = floor
        self.last_pressed = None
        self.jump = False
        self.character_width = character_width
        self.character_height = character_height
        self.wall = None
        self.wall_speed = wall_speed
        
    def __str__(self):
        return str(self.current_x)+" "+str(self.current_y)+" "+str(self.current_xs)+" "+str(self.current_ys)
    
    def give_next_position(self, floor, wall): #fix button 
        #print("The give next position function has run.")
        #print("Button:", button)
        self.floor = floor
        self.wall = wall
        
        print(self.current_y)
        
        if self.current_y <= self.floor:
            self.current_ys = 0
            self.jump = False
        if not self.jump:    
            self.current_xs = 0
        if self.current_y > self.floor:
            self.current_ys -= .5
            
        if self.button != None: 
            #print("AH I SENSE MOVEMENT!")
            if self.current_y == self.floor:
                if self.button.UP:
                    self.current_ys = 10 
                    if self.last_pressed == "RIGHT":
                        self.current_xs = 1
                        self.jump = True
                    elif self.last_pressed == "LEFT":
                        self.current_xs = -1
                        self.jump = True
            if self.button.RIGHT:
                self.current_xs = 1
            elif self.button.LEFT:
                self.current_xs = -1
            self.last_pressed = self.button.last_pressed
            
        if self.wall == "RIGHT" and self.current_xs <= 0:
            self.current_xs = -1*self.wall_speed
        
        self.current_x += self.current_xs
        self.current_y += self.current_ys
        
        if self.current_y < self.floor:
            self.current_y = self.floor
        

In [14]:
###PLATFORMS###
class platform:
    def __init__(self, x1, x2, height):
        self.leftX = x1
        self.rightX = x2
        self.height = height

platform_generation_list = [(120,200,50)]

def generate_list_of_platform_objects(platform_generation_list):
    list_of_platforms = []
    for i in platform_generation_list:
        platform_temp = platform(i[0], i[1], i[2])
        list_of_platforms.append(platform_temp)
    return list_of_platforms

def generate_list_of_platforms_on_screen(list_of_platforms, virtual_window):
    list_of_platforms.sort(key = lambda plat: plat.leftX) ###SORT BY LEFTX
    list_of_platforms_on_screen = []
    for i in list_of_platforms:
        if i.rightX < 0:
            continue
        list_of_platforms_on_screen.append(i)
        if i.leftX > virtual_window[0]:
            break
    return list_of_platforms_on_screen

def blit_platform(platform, window_size, virtual_window):
    rectangle = (platform.leftX, platform.height - 10, platform.rightX-platform.leftX, 10)
    pixel_rectangle = xy_position_to_pixels(rectangle, window_size, virtual_window)
    pygame.draw.rect(screen, BLUE, pixel_rectangle)



###BLOCKS###
class block:
    def __init__(self, x, y, width, height):
        self.leftX = x
        self.rightX = x+width
        self.height = y + height
        self.bot = y
        

def generate_list_of_block_objects(block_generation_list):
    list_of_blocks = []
    for i in block_generation_list:
        block_temp = block(i[0],i[1], i[2], i[3])
        list_of_blocks.append(block_temp)
    return list_of_blocks

def generate_list_of_blocks_on_screen(list_of_blocks, virtual_window):
    list_of_blocks.sort(key = lambda bl: bl.leftX) ###SORT BY LEFTX
    list_of_blocks_on_screen = []
    for i in list_of_blocks:
        if i.rightX < 0:
            continue
        list_of_blocks_on_screen.append(i)
        if i.leftX > virtual_window[0]:
            break
    return list_of_blocks_on_screen

def blit_block(block, window_size, virtual_window):
    rectangle = (block.leftX, block.bot, block.rightX-block.leftX, block.height-block.bot)
    pixel_rectangle = xy_position_to_pixels(rectangle, window_size, virtual_window)
    pygame.draw.rect(screen, BLUE, pixel_rectangle)
    
    
def scroll_everything_left(list_of_platforms, list_of_blocks, world_speed):
    for platform in list_of_platforms:
        platform.rightX -= world_speed
        platform.leftX -= world_speed
    for block in list_of_blocks:
        block.rightX -= world_speed
        block.leftX -= world_speed


In [20]:
def give_floor_value(current_position, blocks_on_screen, platforms_on_screen):
    list_of_floors_under = []
    for i in blocks_on_screen:
        if i.rightX < current_position.current_x:
            continue
        if i.leftX > current_position.current_x:
            break
        if i.leftX < current_position.current_x and current_position.current_x < i.rightX:
            list_of_floors_under.append(i)
            
    for i in platforms_on_screen:
        if i.rightX < current_position.current_x:
            continue
        if i.leftX > current_position.current_x:
            break
        if i.leftX <= current_position.current_x and current_position.current_x <= i.rightX:
            list_of_floors_under.append(i)
    list_of_floors_under.sort(key = lambda structures: structures.height, reverse = True)
    if list_of_floors_under != []:
        list_of_floors_under.sort(key = lambda fl: fl.height, reverse = True)
        #for i in list_of_floors_under:
            
        #return list_of_floors_under[0].height
        print("y= ",current_position.current_y)
        for i in list_of_floors_under:
            if i.height <= current_position.current_y:
                print("floor =",i.height)
                return i.height
    return -100

def give_wall_value(current_position, blocks_on_screen):
    for i in blocks_on_screen:
        if i.rightX < current_position.current_x-1:
            continue
        if i.leftX > current_position.current_x+ current_position.character_width+1:
            break
        if i.leftX == current_position.current_x+current_position.character_width+1:
            return "LEFT"
        if i.rightX == current_position.current_x-1:
            return "RIGHT"
        



In [ ]:
import pygame

virtual_window = (300,200)

pygame.init()
window_size = (pygame.display.Info().current_w -50, pygame.display.Info().current_h - 50)
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("Egg World")
done = False
clock = pygame.time.Clock()
BLUE = (0,0,255)

block_generation_list = [(0,0,300,20), (250,0,50, 50)]
list_of_blocks = generate_list_of_block_objects(block_generation_list)
platform_generation_list = [(120,200,70)]
list_of_platforms = generate_list_of_platform_objects(platform_generation_list)

world_speed = .5

#find floor
current_position = position(current_y= virtual_window[1], wall_speed = world_speed)  



#timer_event = pygame.USEREVENT + 1
#pygame.time.set_timer(timer_event, 250) #makes it so that your not waiting for events to run
while not done:
    scroll_everything_left(list_of_platforms, list_of_blocks, world_speed)
    clock.tick(50)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                done = True
            #print("KEY DOWN!")
            current_position.button = keys(event)
            #print(current_position.button)
            break
        if event.type == pygame.KEYUP:
            #print("KEY UP!")
            current_position.button = None
            break
            
    blocks_on_screen = generate_list_of_blocks_on_screen(list_of_blocks, virtual_window)
    platforms_on_screen = generate_list_of_platforms_on_screen(list_of_platforms, virtual_window)
    
    floor = give_floor_value(current_position, blocks_on_screen, platforms_on_screen)        
    #print("Outside loop sees ", current_position.button)
    wall = give_wall_value(current_position, blocks_on_screen)
    
    current_position.give_next_position(floor = floor, wall = wall)
    screen.fill((0,0,50))
    ##floor
    for i in blocks_on_screen:
        blit_block(i, window_size, virtual_window)
    for i in platforms_on_screen:
        blit_platform(i, window_size, virtual_window)
        
    ##Character
    Rectangle = xy_position_to_pixels(
        (current_position.current_x, current_position.current_y,current_position.character_width,current_position.character_height), window_size, virtual_window)
    pygame.draw.rect(screen, BLUE, Rectangle)
    
    
    pygame.display.flip()
        
        
pygame.quit()



y=  200
floor = 20
200
y=  199.5
floor = 20
199.5
y=  198.5
floor = 20
198.5
y=  197.0
floor = 20
197.0
y=  195.0
floor = 20
195.0
y=  192.5
floor = 20
192.5
y=  189.5
floor = 20
189.5
y=  186.0
floor = 20
186.0
y=  182.0
floor = 20
182.0
y=  177.5
floor = 20
177.5
y=  172.5
floor = 20
172.5
y=  167.0
floor = 20
167.0
y=  161.0
floor = 20
161.0
y=  154.5
floor = 20
154.5
y=  147.5
floor = 20
147.5
y=  140.0
floor = 20
140.0
y=  132.0
floor = 20
132.0
y=  123.5
floor = 20
123.5
y=  114.5
floor = 20
114.5
y=  105.0
floor = 20
105.0
y=  95.0
floor = 20
95.0
y=  84.5
floor = 20
84.5
y=  73.5
floor = 20
73.5
y=  62.0
floor = 20
62.0
y=  50.0
floor = 20
50.0
y=  37.5
floor = 20
37.5
y=  24.5
floor = 20
24.5
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 20
20
y=  20
floor = 2